In [1]:
import pandas as pd
import re

In [2]:
# Define the threshold for toxicity
TOXICITY_THRESHOLD = 0.8
DATA_PATH = '../data/'
# Load the dataset
df = pd.read_csv(DATA_PATH + 'raw/filtered.tsv', delimiter='\t')
df.head()

,id,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
0,0,"If Alkar is flooding her with psychic waste, t...","if Alkar floods her with her mental waste, it ...",0.785171,0.010309,0.014195,0.981983
1,1,Now you're getting nasty.,you're becoming disgusting.,0.749687,0.071429,0.065473,0.999039
2,2,"Well, we could spare your life, for one.","well, we can spare your life.",0.919051,0.268293,0.213313,0.985068
3,3,"Ah! Monkey, you've got to snap out of it.","monkey, you have to wake up.",0.664333,0.309524,0.053362,0.994215
4,4,I've got orders to put her down.,I have orders to kill her.,0.726639,0.181818,0.009402,0.999348


In [3]:
# Build a dictionary of toxic phrases and their replacements
toxic_dict = {}
for index, row in df.iterrows():
    if row['ref_tox'] > TOXICITY_THRESHOLD:
        toxic_dict[row['reference'].lower()] = row['translation'].lower()

In [4]:
# Helper function to maintain the same case
def replace_with_same_case(match_obj):
    match_str = match_obj.group(0)
    replacement = toxic_dict[match_str.lower()]
    if match_str[0].isupper():
        return replacement.capitalize()
    return replacement

# Function to detoxify text
def detoxify(text):
    pattern = re.compile('|'.join(re.escape(key) for key in toxic_dict.keys()), re.IGNORECASE)
    return pattern.sub(replace_with_same_case, text)

In [7]:
# Test the function with a new sentence
test_sentence = "I like that shit."
print(detoxify(test_sentence))

I like this.


In [8]:
# Test the function with a new sentence
test_sentence = "Trying to keep me fucking drugged so I don't know what's going on."
print(detoxify(test_sentence))

You want to fool me so i don't know what's going on.


In [9]:
# Test the function with a new sentence
test_sentence = "Damn,man,i wanted the old football coach,man!"
print(detoxify(test_sentence))

Oh, man, i wanted an old american football game, man!


In [5]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

In [6]:
def is_toxic(text_to_analyze, comments = False):
  # Run the classifier (model and tokenizer)
  results = classifier(text_to_analyze)

  # Interpret the results
  for result in results:
      label = result['label']
      score = result['score']

      # Heuristic for toxicity based on negative sentiment score
      res = label == 'NEGATIVE' and score > TOXICITY_THRESHOLD
      if comments:
        print(f"Label: {label}, Score: {score}")
      return res

In [8]:
import random
data_test = pd.read_csv(DATA_PATH + '/interim/test.csv')
test_texts = data_test['reference'].tolist()
random.shuffle(test_texts)
test_texts = test_texts[:20]

not_toxic = 0

for toxic_text in test_texts:
    text_to_analyze =detoxify(toxic_text)
    toxic = is_toxic(text_to_analyze, False)
    if not toxic:
      not_toxic += 1
print("\nAccuracy:", not_toxic/len(test_texts))


Accuracy: 0.2


In [10]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [11]:
data_test = pd.read_csv(DATA_PATH + 'interim/test.csv').head(100)
test_texts_reference = data_test['reference'].tolist()
test_texts_translation = [[data] for data in data_test['translation'].tolist()]

predictions = [detoxify(toxic_text) for toxic_text in test_texts_reference]
metric.compute(predictions = predictions, references=test_texts_translation)


{'score': 19.40789466901169,
 'counts': [531, 258, 127, 64],
 'totals': [1097, 997, 897, 800],
 'precisions': [48.40474020054695, 25.87763289869609, 14.158305462653288, 8.0],
 'bp': 1.0,
 'sys_len': 1097,
 'ref_len': 994}